# Sdílená paměť (Shared Memory) v Pythonu

Od verze 3.8 nabízí Python modul `multiprocessing.shared_memory`. Umožňuje sdílet data mezi procesy s nulovou režií (Zero-Copy).

To je extrémně užitečné například pro zpracování obrazu, Data Science a Machine Learning (NumPy pole), kdy nechcete kopírovat obrovská pole dat mezi procesy.

## 1. Základní princip (Low-Level)
Sdílená paměť se chová jako pole bajtů (`bytearray`). Jeden proces ji vytvoří, ostatní se připojí.

In [1]:
from multiprocessing.shared_memory import SharedMemory

# 1. Vytvoření sdílené paměti (10 bytů)
try:
    # create=True požádá OS o alokaci nového bloku RAM
    shm = SharedMemory(name="MojeTestovaciPamet", create=True, size=10)
except FileExistsError:
    # Pokud paměť zbyla z minula (např. po pádu skriptu)
    shm = SharedMemory(name="MojeTestovaciPamet")

# 2. Zápis do bufferu (jako do seznamu)
shm.buf[0] = 65  # ASCII 'A'
shm.buf[1] = 66  # ASCII 'B'
shm.buf[2:5] = b'CDE'

# Přečtení
print(f"Data v paměti: {bytes(shm.buf[:5])}")

# 3. Úklid (Velmi důležité!)
shm.close()  # Zavře přístup pro tento proces
shm.unlink() # Smaže paměť ze systému (volá jen vlastník)

Data v paměti: b'ABCDE'


## 2. Sdílení NumPy polí (Zero-Copy)
Toto je "Killer Feature" sdílené paměti v Pythonu. Můžete vytvořit pole v jednom procesu a okamžitě ho vidět v jiném bez kopírování.

V tomto příkladu použijeme `multiprocessing` k vytvoření potomka, který data modifikuje.

In [2]:
import multiprocessing
import time

try:
    import numpy as np
except ImportError:
    print("Pro tento příklad potřebujete knihovnu NumPy (pip install numpy).")
    np = None

def potomek_pracuje(shm_name, shape, dtype):
    # 1. Připojení k existující paměti
    existing_shm = SharedMemory(name=shm_name)
    
    # 2. Vytvoření NumPy pohledu na sdílenou paměť
    np_array = np.ndarray(shape, dtype=dtype, buffer=existing_shm.buf)
    
    # 3. Modifikace dat (násobení -1)
    # Změna se projeví ihned, protože 'np_array' leží ve sdílené RAM
    np_array[:] = np_array * -1
    print("[Potomek] Data změněna.")
    
    existing_shm.close()

if np:
    # Hlavní proces (Rodič)
    data = np.array([1, 2, 3, 4, 5, 100, 200])
    
    # Alokujeme sdílenou paměť přesně pro velikost pole
    shm_np = SharedMemory(create=True, size=data.nbytes)
    
    # Vytvoříme sdílené pole a zkopírujeme do něj data
    shared_arr = np.ndarray(data.shape, dtype=data.dtype, buffer=shm_np.buf)
    shared_arr[:] = data[:]
    
    print(f"[Rodič] Původní pole: {shared_arr}")
    
    # Spustíme proces
    p = multiprocessing.Process(target=potomek_pracuje, 
                                args=(shm_np.name, data.shape, data.dtype))
    p.start()
    p.join()
    
    print(f"[Rodič] Pole po zásahu potomka: {shared_arr}")
    
    # Úklid
    shm_np.close()
    shm_np.unlink()

[Rodič] Původní pole: [  1   2   3   4   5 100 200]
[Potomek] Data změněna.
[Rodič] Pole po zásahu potomka: [  -1   -2   -3   -4   -5 -100 -200]


## 3. Srovnání s `Queue` (Benchmark)
Proč to děláme tak složitě? Protože `Queue` (fronta) je pomalá pro velká data.

In [ ]:
import time
import multiprocessing
from multiprocessing import shared_memory
import sys

# --- Konfigurace ---
# Cílová velikost dat v megabajtech, aby byly rozdíly dobře viditelné.
# "cca 80 MB".
DATA_SIZE_MB = 40
DATA_SIZE_BYTES = DATA_SIZE_MB * 1024 * 1024

# Počet integerů v seznamu, aby serializovaná velikost byla cca 80 MB.
# Pickle ukládá malé integery jako 4-bajtové.
NUM_ELEMENTS = DATA_SIZE_BYTES // 4


def benchmark_local_copy():
    """Benchmark pro kopírování dat v lokální paměti (baseline)."""
    print(f"  (Testovaná data: {DATA_SIZE_MB} MB bytearray)")
    source_data = bytearray(DATA_SIZE_BYTES)
    destination_data = bytearray(DATA_SIZE_BYTES)
    
    start = time.time()
    destination_data[:] = source_data
    end = time.time()
    
    duration = end - start
    throughput = (DATA_SIZE_BYTES / (1024*1024)) / duration if duration > 0 else float('inf')
    print(f"Lokální kopie (bytearray): {duration:.4f} s, Rychlost: {throughput:.2f} MB/s")

def benchmark_queue():
    """Benchmark pro posílání dat přes multiprocessing.Queue."""
    q = multiprocessing.Queue()
    # Seznam integerů, který se musí serializovat (pickle).
    # Je to pomalé a paměťově náročné.
    velka_data = [1] * NUM_ELEMENTS
    
    print(f"  (Testovaná data: seznam {NUM_ELEMENTS / 1_000_000:.1f} mil. integerů, cca {DATA_SIZE_MB} MB po serializaci)")

    start = time.time()
    q.put(velka_data) # Zde dochází k serializaci (Pickle)
    _ = q.get()       # Zde dochází k deserializaci
    end = time.time()
    
    duration = end - start
    throughput = (DATA_SIZE_BYTES / (1024*1024)) / duration if duration > 0 else float('inf')
    print(f"Queue (Pickle):           {duration:.4f} s, Rychlost: {throughput:.2f} MB/s")

def benchmark_shared_memory():
    """Benchmark pro zápis dat do sdílené paměti."""
    print(f"  (Testovaná data: {DATA_SIZE_MB} MB bytearray)")
    # Data, která budeme zapisovat
    source_data = bytearray(DATA_SIZE_BYTES)
    
    # Vytvoření bloku sdílené paměti
    shm = None
    try:
        shm = shared_memory.SharedMemory(create=True, size=DATA_SIZE_BYTES)
        
        start = time.time()
        # Zápis dat do sdílené paměti. shm.buf je memoryview,
        # které umožňuje přímý a rychlý přístup k paměti.
        shm.buf[:] = source_data
        end = time.time()
        
        duration = end - start
        throughput = (DATA_SIZE_BYTES / (1024*1024)) / duration if duration > 0 else float('inf')
        print(f"Shared Memory (zápis):    {duration:.4f} s, Rychlost: {throughput:.2f} MB/s")

    finally:
        if shm:
            # Uvolnění zdrojů
            shm.close()
            shm.unlink() # Odstraní blok sdílené paměti

# --- Spuštění benchmarků ---
# Obalení `if __name__ == "__main__":` je dobrá praxe pro multiprocessing
if __name__ == "__main__":
    multiprocessing.freeze_support() # Pro případ spuštění na Windows jako skript
    
    print(f"--- Benchmark přenosu dat ({DATA_SIZE_MB} MB) ---")

    print("\n[1] Testuji rychlost lokální kopie (baseline)...")
    benchmark_local_copy()

    print("\n[2] Testuji rychlost zápisu do sdílené paměti...")
    benchmark_shared_memory()

    print("\n[3] Testuji rychlost Queue (může chvíli trvat)...")
    benchmark_queue()

    print("\n=== Závěr ===")
    print("Vidíme, že zápis do sdílené paměti (Shared Memory) je řádově rychlejší než přes frontu (Queue).")
    print("Fronta je pomalá kvůli serializaci (pickling) a deserializaci velkého objemu dat.")


--- Benchmark přenosu dat (40 MB) ---

[1] Testuji rychlost lokální kopie (baseline)...
  (Testovaná data: 40 MB bytearray)
Lokální kopie (bytearray): 0.0053 s, Rychlost: 7612.51 MB/s

[2] Testuji rychlost zápisu do sdílené paměti...
  (Testovaná data: 40 MB bytearray)
Shared Memory (zápis):    0.0201 s, Rychlost: 1992.66 MB/s

[3] Testuji rychlost Queue (může chvíli trvat)...
  (Testovaná data: seznam 10.5 mil. integerů, cca 40 MB po serializaci)
Queue (Pickle):           0.3215 s, Rychlost: 124.41 MB/s

=== Závěr ===
Vidíme, že zápis do sdílené paměti (Shared Memory) je řádově rychlejší než přes frontu (Queue).
Rychlost zápisu do sdílené paměti je srovnatelná s rychlostí lokální kopie v RAM.
Fronta je pomalá kvůli serializaci (pickling) a deserializaci velkého objemu dat.
